In [1]:
# Imports we need.
import tensorflow as tf
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()
Modes = tf.estimator.ModeKeys

from define_problem import MimicDischargeSummaries
import subprocess as sp

W0731 17:34:33.837530 139662910273344 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/utils/expert_utils.py:68: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0731 17:34:34.360372 139662910273344 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0731 17:34:36.136854 139662910273344 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/utils/adafactor.py:27: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0731 17:34:36.138069 139662910273344 deprecation_wrapper

In [2]:
# Fetch the problem
ende_problem = problems.problem("mimic_discharge_summaries")

# Get the encoders from the problem
data_dir = os.path.expanduser("../data/t2t_experiments/full_context/data/")
encoders = ende_problem.feature_encoders(data_dir)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
    """Input str to features dict, ready for inference"""
    inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
    return {"inputs": batch_inputs}

def decode(integers):
    """List of ints to str"""
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(np.squeeze(integers))

W0731 17:34:36.745614 139662910273344 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/data_generators/text_encoder.py:938: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0731 17:34:36.746424 139662910273344 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/data_generators/text_encoder.py:940: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



In [3]:
# Copy the pretrained checkpoint locally
ckpt_path=u'../data/t2t_experiments/full_context/output/model.ckpt-5000'

In [4]:
# Create hparams and the model
model_name = "transformer"
hparams_set = "transformer_base"

hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="mimic_discharge_summaries")

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
translate_model = registry.model(model_name)(hparams, Modes.EVAL)

W0731 17:34:37.037321 139662910273344 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/layers/common_layers.py:94: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0731 17:34:37.682513 139662910273344 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/utils/t2t_model.py:243: The name tf.summary.text is deprecated. Please use tf.compat.v1.summary.text instead.



In [6]:
# Restore and translate!
def translate(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = translate_model.infer(encoded_inputs, beam_size=4, alpha=0.6)["outputs"]
    return decode(model_output)

inputs = sp.getoutput('head -1 ../data/preprocessed/src-test.txt')
outputs = translate(inputs)

#print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)

Outputs: admission date : [ 2143/8/14 ] discharge date : [ 2143/8/23 ] <PAR> <PAR> date of birth : [ 2077/4/24 ] sex : m <PAR> <PAR> service : surgery <PAR> <PAR> allergies : <PAR> patient recorded as having no known allergies to drugs <PAR> <PAR> attending : [ first name3 ( lf ) 148 ] <PAR> chief complaint : <PAR> abdominal pain <PAR> <PAR> major surgical or invasive procedure : <PAR> none <PAR> <PAR> history of present illness : <PAR> mr [ known lastname ] is a 68 year old male with a history of <PAR> hypertension , hyperlipidemia , and recent <PAR> abdominal pain who presents with abdominal pain and <PAR> abdominal pain he was in his usual state of health until <PAR> approximately 1 week ago when he developed nausea and <PAR> vomiting he was brought to the ed where he was <PAR> found to have a lipase of 19 and a lipase of <PAR> 19 he was given 2 liters of normal saline and was <PAR> transferred to the [ hospital1 18 ] 
